<h1 align="center">OpenMP part 2</h1> 

## Exercise 1:

Using your code 2D non linear convection equation from the `numba_pyccel_assignments`:
1. Parallelize the 2D non linear convection equation
2. Compare cpu time using different number of threads (Plot a figure of cpu time according to number of threads)

In [1]:
# write your code here

def solve_2d_nonlinearconv_pyccel(u:'float[:,:]', un:'float[:,:]', v:'float[:,:]', vn:'float[:,:]', nt:'int', dt:'float', dx:'float', dy:'float',c:'int',k:'int'):

    ###Assign initial conditions
    ##set hat function I.C. : u(.5<=x<=1 && .5<=y<=1 ) is 2
    u[int(.5 / dy):int(1 / dy + 1), int(.5 / dx):int(1 / dx + 1)] = 2
    ##set hat function I.C. : v(.5<=x<=1 && .5<=y<=1 ) is 2
    v[int(.5 / dy):int(1 / dy + 1), int(.5 / dx):int(1 / dx + 1)] = 2
    row, col = u.shape
    
    #fill the update of u and v
    from pyccel.stdlib.internal.openmp  import omp_get_thread_num,omp_set_num_threads,omp_get_num_threads
    #$ omp parallel
    omp_set_num_threads(k)
    #$ omp for 
    for t in range(nt):
        un[:,:]=u[:,:]
        vn[:,:]=v[:,:]
        # mise a jour       
        for i in range(1,row-1):
            for j in range(1,col-1):
                u[i,j]=un[i,j]- un[i,j]*(dt/dx)*(un[i,j]-un[i-1,j]) - vn[i,j]*(dt/dy)*(un[i,j]-un[i,j-1]) 
                v[i,j]=vn[i,j]- un[i,j]*(dt/dx)*(vn[i,j]-vn[i-1,j]) - vn[i,j]*(dt/dy)*(vn[i,j]-vn[i,j-1]) 
     
    #$ omp end parallel    
    return 0


In [4]:
from pyccel.epyccel import epyccel
from pyccel.decorators import types

f1=epyccel(solve_2d_nonlinearconv_pyccel)
import timeit 





## Exercise 2:

Using your code Navier Stokes equation from the `numba_pyccel_assignments`:
1. Pyccelize the Navier Stokes equation (if it's not done).
2. Parallelize it using OpenMP.
3. Compare cpu time using different number of threads (Plot a figure of cpu time according to number of threads)

In [1]:
# write your code here

